In [1]:
!pip install pandas-profiling==2.*


In [2]:
import sys
import itertools
import numpy as np 
import pandas as pd
import re
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from feature_engine.encoding import OneHotEncoder as fe_OneHotEncoder
#from pandas_profiling import ProfileReport

In [48]:
# def wrangle(df3):
#     df3['artist_ids'] = df3['artist_ids'].apply(lambda x: re.findall(r"'([^']*)'", x))
#     df3['ID'] = df['artist_ids'].apply(lambda x: "".join(x))
#     df3.drop('artist_ids')
#     return df3
    
    

In [3]:
df = pd.read_csv('tracks_features.csv')

In [4]:
df.columns

Index(['id', 'name', 'album', 'album_id', 'artists', 'artist_ids',
       'track_number', 'disc_number', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'time_signature', 'year', 'release_date'],
      dtype='object')

In [46]:
df.head();

In [5]:
df['artist_ids'].values[0]

"['2d0hyoQ5ynDBnkvAbJKORj']"

In [6]:
df['artist_ids'] = df['artist_ids'].apply(lambda x: re.findall(r"'([^']*)'", x))

In [7]:
df['artist_ids'].values[0]


['2d0hyoQ5ynDBnkvAbJKORj']

In [8]:
df['artist_ids'] = df['artist_ids'].apply(lambda x: "".join(x))

In [9]:
df['artist_ids'].values[0]

'2d0hyoQ5ynDBnkvAbJKORj'

In [10]:
df.head()

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
0,7lmeHLHBe4nmXzuXc0HDjk,Testify,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],2d0hyoQ5ynDBnkvAbJKORj,1,1,False,0.470,...,0.0727,0.02610,0.000011,0.3560,0.503,117.906,210133,4.0,1999,1999-11-02
1,1wsRitfRRtWyEapl0q22o8,Guerrilla Radio,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],2d0hyoQ5ynDBnkvAbJKORj,2,1,True,0.599,...,0.1880,0.01290,0.000071,0.1550,0.489,103.680,206200,4.0,1999,1999-11-02
2,1hR0fIFK2qRG3f3RF70pb7,Calm Like a Bomb,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],2d0hyoQ5ynDBnkvAbJKORj,3,1,False,0.315,...,0.4830,0.02340,0.000002,0.1220,0.370,149.749,298893,4.0,1999,1999-11-02
3,2lbASgTSoDO7MTuLAXlTW0,Mic Check,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],2d0hyoQ5ynDBnkvAbJKORj,4,1,True,0.440,...,0.2370,0.16300,0.000004,0.1210,0.574,96.752,213640,4.0,1999,1999-11-02
4,1MQTmpYOZ6fcMQc56Hdo7T,Sleep Now In the Fire,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],2d0hyoQ5ynDBnkvAbJKORj,5,1,False,0.426,...,0.0701,0.00162,0.105000,0.0789,0.539,127.059,205600,4.0,1999,1999-11-02


In [11]:
df1 = pd.read_csv('artist_genres.csv')

In [12]:
df1.head()

,Unnamed: 0,ID,Genres
0,0,2d0hyoQ5ynDBnkvAbJKORj,"['alternative metal', 'alternative rock', 'con..."
1,1,77tT1kLj6mCWtFNqiOmP9H,"['album rock', 'classic rock', 'folk rock', 'm..."
2,2,2U6gqwyl9F33YxawnFrZG7,"['dance pop', 'europop', 'new wave pop']"
3,3,4sh4MHP7lhrSUakxwZzwqz,"['canadian singer-songwriter', 'classic canadi..."
4,4,1KsASRNugxU85T0u6zSg32,"['art pop', 'ectofolk', 'folk', 'lilith', 'mel..."


In [13]:
df1['ID'].values[0]

'2d0hyoQ5ynDBnkvAbJKORj'

In [14]:
df1.columns

Index(['Unnamed: 0', 'ID', 'Genres'], dtype='object')

In [15]:
df1.drop('Unnamed: 0', axis = 1,inplace = True)

In [16]:
df1.dtypes

ID        object
Genres    object
dtype: object

In [19]:
df.columns

Index(['id', 'name', 'album', 'album_id', 'artists', 'artist_ids',
       'track_number', 'disc_number', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'time_signature', 'year', 'release_date'],
      dtype='object')

In [20]:
df1.ID.isnull().sum()

0

In [67]:
# df.artist_ids = df.artist_ids.astype(str)
# df1.ID = df1.ID.astype(str)

In [17]:
df3 = pd.DataFrame()

In [18]:
df3 = df.merge(df1, how="left",left_on = 'artist_ids', right_on = 'ID')

In [73]:
df.columns

Index(['id', 'name', 'album', 'album_id', 'artists', 'artist_ids',
       'track_number', 'disc_number', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'time_signature', 'year', 'release_date'],
      dtype='object')

In [19]:
df3.head()

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date,ID,Genres
0,7lmeHLHBe4nmXzuXc0HDjk,Testify,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],2d0hyoQ5ynDBnkvAbJKORj,1,1,False,0.470,...,0.000011,0.3560,0.503,117.906,210133,4.0,1999,1999-11-02,2d0hyoQ5ynDBnkvAbJKORj,"['alternative metal', 'alternative rock', 'con..."
1,1wsRitfRRtWyEapl0q22o8,Guerrilla Radio,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],2d0hyoQ5ynDBnkvAbJKORj,2,1,True,0.599,...,0.000071,0.1550,0.489,103.680,206200,4.0,1999,1999-11-02,2d0hyoQ5ynDBnkvAbJKORj,"['alternative metal', 'alternative rock', 'con..."
2,1hR0fIFK2qRG3f3RF70pb7,Calm Like a Bomb,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],2d0hyoQ5ynDBnkvAbJKORj,3,1,False,0.315,...,0.000002,0.1220,0.370,149.749,298893,4.0,1999,1999-11-02,2d0hyoQ5ynDBnkvAbJKORj,"['alternative metal', 'alternative rock', 'con..."
3,2lbASgTSoDO7MTuLAXlTW0,Mic Check,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],2d0hyoQ5ynDBnkvAbJKORj,4,1,True,0.440,...,0.000004,0.1210,0.574,96.752,213640,4.0,1999,1999-11-02,2d0hyoQ5ynDBnkvAbJKORj,"['alternative metal', 'alternative rock', 'con..."
4,1MQTmpYOZ6fcMQc56Hdo7T,Sleep Now In the Fire,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],2d0hyoQ5ynDBnkvAbJKORj,5,1,False,0.426,...,0.105000,0.0789,0.539,127.059,205600,4.0,1999,1999-11-02,2d0hyoQ5ynDBnkvAbJKORj,"['alternative metal', 'alternative rock', 'con..."


In [20]:
df3.shape

(1204025, 26)

In [21]:
df3['artists'].values[0]

"['Rage Against The Machine']"

In [23]:
def explicit_converter(x):
    if x:
        return 1 
    else:
        return 0

In [24]:
def decade_function(x):
    if x in range(1900,1991):
        return '1900-1990'
    elif x in range(1991,1996):
        return '1991-1995'
    elif x in range(1996,2001):
        return '1996-2000'
    elif x in range(2001,2006):
        return '2001-2005'
    elif x == 2006:
        return '2006' 
    elif x == 2007:
        return '2007'
    elif x == 2008:
        return '2008'
    elif x == 2009:
        return '2009'
    elif x == 2010:
        return '2010'
    elif x == 2011:
        return '2011'
    elif x == 2012:
        return '2012'
    elif x == 2013:
        return '2013'
    elif x == 2014:
        return '2014'
    elif x == 2015:
        return '2015'
    elif x == 2016:
        return '2016'
    elif x == 2017:
        return '2017'
    elif x == 2018:
        return '2018'
    elif x == 2019:
        return '2019'
    elif x == 2020:
        return '2020'
    elif x == 2021:
        return '2021'
        

In [25]:
def wrangle(df3):
    df3['artists_new'] = df3['artists'].apply(lambda x: re.findall(r"'([^']*)'", x))
    df3['artists_new2'] = df3['artists'].apply(lambda x: re.findall('\"(.*?)\"',x))
    df3['artists_final'] = np.where(df3['artists_new'].apply(lambda x: not x), df3['artists_new2'], df3['artists_new'])
    df3['artists_song'] = df3.apply(lambda row: row['artists_final'][0]+row['name'],axis = 1)
    df3.drop_duplicates('artists_song',inplace = True)
    cols_to_drop = ['id', 'album_id','artists', 'artist_ids', 'track_number', 'disc_number', 'release_date', 'ID', 'artists_new', 'artists_new2', 'artists_song', 'duration_ms']
    df3.drop(columns = cols_to_drop, inplace = True)
    df3['Genres'] = df3['Genres'].apply(lambda x: [re.sub(' ','_',i) for i in re.findall(r"'([^']*)'", str(x))])
    df3['Genres'] = df3['Genres'].apply(lambda x: x if isinstance(x, list) else []) 
    df3['explicit'] = df3['explicit'].apply(explicit_converter)
    df3.rename(columns={"Genres": "genres"}, inplace = True)
    df3['year'] = df3['year'].replace(0, pd.np.nan)
    df3.dropna(subset = ['year'], inplace = True)
    return df3
    

In [26]:
df3.columns #before cleaning 

Index(['id', 'name', 'album', 'album_id', 'artists', 'artist_ids',
       'track_number', 'disc_number', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'time_signature', 'year', 'release_date', 'ID', 'Genres'],
      dtype='object')

In [27]:
wrangle(df3)

<ipython-input-25-73e8a2036be0>:13: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  df3['year'] = df3['year'].replace(0, pd.np.nan)


,name,album,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,genres,artists_final
0,Testify,The Battle Of Los Angeles,0,0.470,0.978,7,-5.399,1,0.0727,0.02610,0.000011,0.3560,0.503,117.906,4.0,1999.0,"[alternative_metal, alternative_rock, consciou...",[Rage Against The Machine]
1,Guerrilla Radio,The Battle Of Los Angeles,1,0.599,0.957,11,-5.764,1,0.1880,0.01290,0.000071,0.1550,0.489,103.680,4.0,1999.0,"[alternative_metal, alternative_rock, consciou...",[Rage Against The Machine]
2,Calm Like a Bomb,The Battle Of Los Angeles,0,0.315,0.970,7,-5.424,1,0.4830,0.02340,0.000002,0.1220,0.370,149.749,4.0,1999.0,"[alternative_metal, alternative_rock, consciou...",[Rage Against The Machine]
3,Mic Check,The Battle Of Los Angeles,1,0.440,0.967,11,-5.830,0,0.2370,0.16300,0.000004,0.1210,0.574,96.752,4.0,1999.0,"[alternative_metal, alternative_rock, consciou...",[Rage Against The Machine]
4,Sleep Now In the Fire,The Battle Of Los Angeles,0,0.426,0.929,2,-6.729,1,0.0701,0.00162,0.105000,0.0789,0.539,127.059,4.0,1999.0,"[alternative_metal, alternative_rock, consciou...",[Rage Against The Machine]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1204020,Gospel of Juke,Notch - EP,0,0.264,0.966,5,-6.970,0,0.0672,0.00935,0.002240,0.3370,0.415,159.586,4.0,2014.0,[slovak_electronic],[FVLCRVM]
1204021,Prism Visions,Notch - EP,0,0.796,0.701,11,-6.602,0,0.0883,0.10400,0.644000,0.0749,0.781,121.980,4.0,2014.0,[slovak_electronic],[FVLCRVM]
1204022,Tokyo 360,Notch - EP,0,0.785,0.796,9,-5.960,0,0.0564,0.03040,0.918000,0.0664,0.467,121.996,4.0,2014.0,[slovak_electronic],[FVLCRVM]
1204023,Yummy!,Notch - EP,0,0.665,0.856,6,-6.788,0,0.0409,0.00007,0.776000,0.1170,0.227,124.986,4.0,2014.0,[slovak_electronic],[FVLCRVM]


In [162]:
df3.Genres.values[1000]

"['alternative metal', 'industrial metal', 'industrial rock', 'nu metal', 'rap metal', 'rap rock', 'slayer']"

In [28]:
df3.genres.values[1000]

['alternative_metal',
 'industrial_metal',
 'industrial_rock',
 'nu_metal',
 'rap_metal',
 'rap_rock',
 'slayer']

In [29]:
df3.artists_final.values[0]

['Rage Against The Machine']

In [30]:
df3.name.values[0]

'Testify'

In [37]:
df3.isnull().sum()

name                0
album               0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
year                0
genres              0
artists_final       0
dtype: int64

In [170]:
df3.isnull().sum()

name                0
album               0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
duration_ms         0
time_signature      0
year                0
Genres              0
artists_final       0
dtype: int64

In [38]:
df3.genres.values[0]

['alternative_metal',
 'alternative_rock',
 'conscious_hip_hop',
 'funk_metal',
 'nu_metal',
 'political_hip_hop',
 'rap_metal',
 'rap_rock',
 'rock']

In [194]:
df3.to_csv('spotify_clean_v1.csv')

In [230]:
df3.Genres.values[0]

['alternative_metal',
 'alternative_rock',
 'conscious_hip_hop',
 'funk_metal',
 'nu_metal',
 'political_hip_hop',
 'rap_metal',
 'rap_rock',
 'rock']

In [31]:
df3.explicit.nunique()

2

In [32]:
df3.explicit.value_counts()

0    1046825
1      79351
Name: explicit, dtype: int64

In [33]:
float_cols = df3.dtypes[df3.dtypes == 'float64'].index.values

In [34]:
float_cols

array(['danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature', 'year'], dtype=object)

In [35]:
df3.year.value_counts()

2020.0    65403
2019.0    61332
2006.0    54353
2007.0    53841
2018.0    51546
          ...  
1920.0       24
1946.0       23
1908.0       19
1935.0       19
1900.0       13
Name: year, Length: 100, dtype: int64

In [44]:
df3['year'] = df3['year'].replace(0, pd.np.nan)

<ipython-input-44-d4c3c35fec02>:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  df3['year'] = df3['year'].replace(0, pd.np.nan)


In [36]:
df3.isnull().sum()

name                0
album               0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
year                0
genres              0
artists_final       0
dtype: int64

In [47]:
df3.year.isnull().sum() #years with value 0 removed

0

In [37]:
df3.shape

(1126176, 18)

In [291]:
df3.year.isnull().sum()

0

In [292]:
df3.shape

(1126176, 18)

In [38]:
df4 = df3.copy()

In [39]:
df4.head()

,name,album,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,genres,artists_final
0,Testify,The Battle Of Los Angeles,0,0.470,0.978,7,-5.399,1,0.0727,0.02610,0.000011,0.3560,0.503,117.906,4.0,1999.0,"[alternative_metal, alternative_rock, consciou...",[Rage Against The Machine]
1,Guerrilla Radio,The Battle Of Los Angeles,1,0.599,0.957,11,-5.764,1,0.1880,0.01290,0.000071,0.1550,0.489,103.680,4.0,1999.0,"[alternative_metal, alternative_rock, consciou...",[Rage Against The Machine]
2,Calm Like a Bomb,The Battle Of Los Angeles,0,0.315,0.970,7,-5.424,1,0.4830,0.02340,0.000002,0.1220,0.370,149.749,4.0,1999.0,"[alternative_metal, alternative_rock, consciou...",[Rage Against The Machine]
3,Mic Check,The Battle Of Los Angeles,1,0.440,0.967,11,-5.830,0,0.2370,0.16300,0.000004,0.1210,0.574,96.752,4.0,1999.0,"[alternative_metal, alternative_rock, consciou...",[Rage Against The Machine]
4,Sleep Now In the Fire,The Battle Of Los Angeles,0,0.426,0.929,2,-6.729,1,0.0701,0.00162,0.105000,0.0789,0.539,127.059,4.0,1999.0,"[alternative_metal, alternative_rock, consciou...",[Rage Against The Machine]


In [40]:
df4['age'] = df4['year'].apply(np.int64) #added new age feature to bucket years

In [41]:
df4.head()

,name,album,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,genres,artists_final,age
0,Testify,The Battle Of Los Angeles,0,0.470,0.978,7,-5.399,1,0.0727,0.02610,0.000011,0.3560,0.503,117.906,4.0,1999.0,"[alternative_metal, alternative_rock, consciou...",[Rage Against The Machine],1999
1,Guerrilla Radio,The Battle Of Los Angeles,1,0.599,0.957,11,-5.764,1,0.1880,0.01290,0.000071,0.1550,0.489,103.680,4.0,1999.0,"[alternative_metal, alternative_rock, consciou...",[Rage Against The Machine],1999
2,Calm Like a Bomb,The Battle Of Los Angeles,0,0.315,0.970,7,-5.424,1,0.4830,0.02340,0.000002,0.1220,0.370,149.749,4.0,1999.0,"[alternative_metal, alternative_rock, consciou...",[Rage Against The Machine],1999
3,Mic Check,The Battle Of Los Angeles,1,0.440,0.967,11,-5.830,0,0.2370,0.16300,0.000004,0.1210,0.574,96.752,4.0,1999.0,"[alternative_metal, alternative_rock, consciou...",[Rage Against The Machine],1999
4,Sleep Now In the Fire,The Battle Of Los Angeles,0,0.426,0.929,2,-6.729,1,0.0701,0.00162,0.105000,0.0789,0.539,127.059,4.0,1999.0,"[alternative_metal, alternative_rock, consciou...",[Rage Against The Machine],1999


In [53]:
df4.dtypes

name                 object
album                object
explicit              int64
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
time_signature      float64
year                float64
genres               object
artists_final        object
age                   int64
dtype: object

In [42]:
df4['age'] = df4['age'].apply(decade_function)

In [43]:
df4.head()

,name,album,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,genres,artists_final,age
0,Testify,The Battle Of Los Angeles,0,0.470,0.978,7,-5.399,1,0.0727,0.02610,0.000011,0.3560,0.503,117.906,4.0,1999.0,"[alternative_metal, alternative_rock, consciou...",[Rage Against The Machine],1996-2000
1,Guerrilla Radio,The Battle Of Los Angeles,1,0.599,0.957,11,-5.764,1,0.1880,0.01290,0.000071,0.1550,0.489,103.680,4.0,1999.0,"[alternative_metal, alternative_rock, consciou...",[Rage Against The Machine],1996-2000
2,Calm Like a Bomb,The Battle Of Los Angeles,0,0.315,0.970,7,-5.424,1,0.4830,0.02340,0.000002,0.1220,0.370,149.749,4.0,1999.0,"[alternative_metal, alternative_rock, consciou...",[Rage Against The Machine],1996-2000
3,Mic Check,The Battle Of Los Angeles,1,0.440,0.967,11,-5.830,0,0.2370,0.16300,0.000004,0.1210,0.574,96.752,4.0,1999.0,"[alternative_metal, alternative_rock, consciou...",[Rage Against The Machine],1996-2000
4,Sleep Now In the Fire,The Battle Of Los Angeles,0,0.426,0.929,2,-6.729,1,0.0701,0.00162,0.105000,0.0789,0.539,127.059,4.0,1999.0,"[alternative_metal, alternative_rock, consciou...",[Rage Against The Machine],1996-2000


In [44]:
df4.age.unique()

array(['1996-2000', '1991-1995', '2018', '2001-2005', '2011', '1900-1990',
       '2017', '2010', '2019', '2007', '2020', '2015', '2016', '2009',
       '2014', '2008', '2012', '2013', '2006'], dtype=object)

In [45]:
ohe_enc = fe_OneHotEncoder(
    top_categories=None,
    variables=['age'], drop_last=True)  # to return k-1, false to return k


ohe_enc.fit(df4.fillna('Missing'))
#OHE for new age feature 

OneHotEncoder(drop_last=True, variables=['age'])

In [48]:
df4 = ohe_enc.transform(df4.fillna('Missing'))

In [49]:
df4.head()

,name,album,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,...,age_2019,age_2007,age_2020,age_2015,age_2016,age_2009,age_2014,age_2008,age_2012,age_2013
0,Testify,The Battle Of Los Angeles,0,0.470,0.978,7,-5.399,1,0.0727,0.02610,...,0,0,0,0,0,0,0,0,0,0
1,Guerrilla Radio,The Battle Of Los Angeles,1,0.599,0.957,11,-5.764,1,0.1880,0.01290,...,0,0,0,0,0,0,0,0,0,0
2,Calm Like a Bomb,The Battle Of Los Angeles,0,0.315,0.970,7,-5.424,1,0.4830,0.02340,...,0,0,0,0,0,0,0,0,0,0
3,Mic Check,The Battle Of Los Angeles,1,0.440,0.967,11,-5.830,0,0.2370,0.16300,...,0,0,0,0,0,0,0,0,0,0
4,Sleep Now In the Fire,The Battle Of Los Angeles,0,0.426,0.929,2,-6.729,1,0.0701,0.00162,...,0,0,0,0,0,0,0,0,0,0


In [61]:
df4.age.value_counts(normalize=True)

2001-2005    0.154209
1996-2000    0.095441
2020         0.058075
2019         0.054460
1900-1990    0.051354
1991-1995    0.048953
2006         0.048263
2007         0.047809
2018         0.045771
2008         0.042862
2017         0.042503
2013         0.040967
2016         0.039784
2015         0.039264
2009         0.038761
2011         0.038494
2010         0.038016
2012         0.037658
2014         0.037354
Name: age, dtype: float64

In [55]:
df4.head()

,name,album,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,...,age_2019,age_2007,age_2020,age_2015,age_2016,age_2009,age_2014,age_2008,age_2012,age_2013
0,Testify,The Battle Of Los Angeles,0,0.470,0.978,7,-5.399,1,0.0727,0.02610,...,0,0,0,0,0,0,0,0,0,0
1,Guerrilla Radio,The Battle Of Los Angeles,1,0.599,0.957,11,-5.764,1,0.1880,0.01290,...,0,0,0,0,0,0,0,0,0,0
2,Calm Like a Bomb,The Battle Of Los Angeles,0,0.315,0.970,7,-5.424,1,0.4830,0.02340,...,0,0,0,0,0,0,0,0,0,0
3,Mic Check,The Battle Of Los Angeles,1,0.440,0.967,11,-5.830,0,0.2370,0.16300,...,0,0,0,0,0,0,0,0,0,0
4,Sleep Now In the Fire,The Battle Of Los Angeles,0,0.426,0.929,2,-6.729,1,0.0701,0.00162,...,0,0,0,0,0,0,0,0,0,0


In [58]:
#creating vectorized genre features based on frequency of genre. More frequent genre = more weight
tf_idf = TfidfVectorizer()
tfidf_matrix =  tf_idf.fit_transform(df4['genres'].apply(lambda x: " ".join(x)))
genre_df = pd.DataFrame(tfidf_matrix.toarray())
genre_df.columns = ['genre' + "|" + i for i in tf_idf.get_feature_names()]
genre_df.reset_index(drop = True, inplace=True)
    

In [59]:
df4.columns

Index(['name', 'album', 'explicit', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'time_signature', 'year', 'genres',
       'artists_final', 'age_1996-2000', 'age_1991-1995', 'age_2018',
       'age_2001-2005', 'age_2011', 'age_1900-1990', 'age_2017', 'age_2010',
       'age_2019', 'age_2007', 'age_2020', 'age_2015', 'age_2016', 'age_2009',
       'age_2014', 'age_2008', 'age_2012', 'age_2013'],
      dtype='object')

In [60]:
genre_df.head()

,genre|21st_century_classical,genre|48g,genre|5th_wave_emo,genre|_brasileira,genre|_hip_hop,genre|_house,genre|a_cappella,genre|aarhus_indie,genre|aberdeen_indie,genre|abstract,...,genre|zhongguo_feng,genre|zillertal,genre|zim_urban_groove,genre|zimdancehall,genre|zolo,genre|zouglou,genre|zouk,genre|zouk_riddim,genre|zurich_indie,genre|zydeco
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
genre_df['genre|alternative_rock'].value_counts()

0.000000    1114257
0.201082        708
0.210507        678
0.244773        296
0.479359        296
             ...   
0.383642          1
0.291689          1
0.323510          1
0.275375          1
0.533334          1
Name: genre|alternative_rock, Length: 257, dtype: int64

In [61]:
genre_df.columns

Index(['genre|21st_century_classical', 'genre|48g', 'genre|5th_wave_emo',
       'genre|_brasileira', 'genre|_hip_hop', 'genre|_house',
       'genre|a_cappella', 'genre|aarhus_indie', 'genre|aberdeen_indie',
       'genre|abstract',
       ...
       'genre|zhongguo_feng', 'genre|zillertal', 'genre|zim_urban_groove',
       'genre|zimdancehall', 'genre|zolo', 'genre|zouglou', 'genre|zouk',
       'genre|zouk_riddim', 'genre|zurich_indie', 'genre|zydeco'],
      dtype='object', length=4411)

In [67]:
genre_df.head()

,genre|21st_century_classical,genre|48g,genre|5th_wave_emo,genre|_brasileira,genre|_hip_hop,genre|_house,genre|a_cappella,genre|aarhus_indie,genre|aberdeen_indie,genre|abstract,...,genre|zhongguo_feng,genre|zillertal,genre|zim_urban_groove,genre|zimdancehall,genre|zolo,genre|zouglou,genre|zouk,genre|zouk_riddim,genre|zurich_indie,genre|zydeco
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
df4.head()

,name,album,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,...,age_2019,age_2007,age_2020,age_2015,age_2016,age_2009,age_2014,age_2008,age_2012,age_2013
0,Testify,The Battle Of Los Angeles,0,0.470,0.978,7,-5.399,1,0.0727,0.02610,...,0,0,0,0,0,0,0,0,0,0
1,Guerrilla Radio,The Battle Of Los Angeles,1,0.599,0.957,11,-5.764,1,0.1880,0.01290,...,0,0,0,0,0,0,0,0,0,0
2,Calm Like a Bomb,The Battle Of Los Angeles,0,0.315,0.970,7,-5.424,1,0.4830,0.02340,...,0,0,0,0,0,0,0,0,0,0
3,Mic Check,The Battle Of Los Angeles,1,0.440,0.967,11,-5.830,0,0.2370,0.16300,...,0,0,0,0,0,0,0,0,0,0
4,Sleep Now In the Fire,The Battle Of Los Angeles,0,0.426,0.929,2,-6.729,1,0.0701,0.00162,...,0,0,0,0,0,0,0,0,0,0


In [357]:
df4.columns

Index(['name', 'album', 'explicit', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'time_signature', 'year', 'genres',
       'artists_final', 'age_1996-2000', 'age_1991-1995', 'age_2018',
       'age_2001-2005', 'age_2011', 'age_1900-1990', 'age_2017', 'age_2010',
       'age_2019', 'age_2007', 'age_2020', 'age_2015', 'age_2016', 'age_2009',
       'age_2014', 'age_2008', 'age_2012', 'age_2013'],
      dtype='object')

In [353]:
df4.genres

0          [alternative_metal, alternative_rock, consciou...
1          [alternative_metal, alternative_rock, consciou...
2          [alternative_metal, alternative_rock, consciou...
3          [alternative_metal, alternative_rock, consciou...
4          [alternative_metal, alternative_rock, consciou...
                                 ...                        
1204020                                  [slovak_electronic]
1204021                                  [slovak_electronic]
1204022                                  [slovak_electronic]
1204023                                  [slovak_electronic]
1204024                                  [slovak_electronic]
Name: genres, Length: 1126176, dtype: object

In [62]:
float_cols

array(['danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature', 'year'], dtype=object)

In [67]:
float_cols_new = np.delete(float_cols, 10)

In [68]:
float_cols_new

array(['danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature'], dtype=object)

In [69]:
#feature scailing for float columns in df3
floats = df[float_cols_new].reset_index(drop=True)
scaler = MinMaxScaler()
scaled_float_df = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns)

In [70]:
scaled_float_df

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,0.470,0.978,0.812104,0.075026,0.026205,0.000011,0.3560,0.503,0.473644,0.8
1,0.599,0.957,0.806675,0.194014,0.012952,0.000071,0.1550,0.489,0.416496,0.8
2,0.315,0.970,0.811732,0.498452,0.023494,0.000002,0.1220,0.370,0.601561,0.8
3,0.440,0.967,0.805694,0.244582,0.163655,0.000004,0.1210,0.574,0.388665,0.8
4,0.426,0.929,0.792322,0.072343,0.001627,0.105000,0.0789,0.539,0.510412,0.8
...,...,...,...,...,...,...,...,...,...,...
1204020,0.264,0.966,0.788738,0.069350,0.009388,0.002240,0.3370,0.415,0.641078,0.8
1204021,0.796,0.701,0.794211,0.091125,0.104418,0.644000,0.0749,0.781,0.490009,0.8
1204022,0.785,0.796,0.803760,0.058204,0.030522,0.918000,0.0664,0.467,0.490074,0.8
1204023,0.665,0.856,0.791445,0.042208,0.000070,0.776000,0.1170,0.227,0.502085,0.8


In [380]:
df3.danceability.max()

1.0

In [71]:
df4.select_dtypes(include=['float64'])

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year
0,0.470,0.978,-5.399,0.0727,0.02610,0.000011,0.3560,0.503,117.906,4.0,1999.0
1,0.599,0.957,-5.764,0.1880,0.01290,0.000071,0.1550,0.489,103.680,4.0,1999.0
2,0.315,0.970,-5.424,0.4830,0.02340,0.000002,0.1220,0.370,149.749,4.0,1999.0
3,0.440,0.967,-5.830,0.2370,0.16300,0.000004,0.1210,0.574,96.752,4.0,1999.0
4,0.426,0.929,-6.729,0.0701,0.00162,0.105000,0.0789,0.539,127.059,4.0,1999.0
...,...,...,...,...,...,...,...,...,...,...,...
1204020,0.264,0.966,-6.970,0.0672,0.00935,0.002240,0.3370,0.415,159.586,4.0,2014.0
1204021,0.796,0.701,-6.602,0.0883,0.10400,0.644000,0.0749,0.781,121.980,4.0,2014.0
1204022,0.785,0.796,-5.960,0.0564,0.03040,0.918000,0.0664,0.467,121.996,4.0,2014.0
1204023,0.665,0.856,-6.788,0.0409,0.00007,0.776000,0.1170,0.227,124.986,4.0,2014.0


In [71]:
#df6 = df5.copy()

In [72]:
df4.head()

,name,album,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,...,age_2019,age_2007,age_2020,age_2015,age_2016,age_2009,age_2014,age_2008,age_2012,age_2013
0,Testify,The Battle Of Los Angeles,0,0.470,0.978,7,-5.399,1,0.0727,0.02610,...,0,0,0,0,0,0,0,0,0,0
1,Guerrilla Radio,The Battle Of Los Angeles,1,0.599,0.957,11,-5.764,1,0.1880,0.01290,...,0,0,0,0,0,0,0,0,0,0
2,Calm Like a Bomb,The Battle Of Los Angeles,0,0.315,0.970,7,-5.424,1,0.4830,0.02340,...,0,0,0,0,0,0,0,0,0,0
3,Mic Check,The Battle Of Los Angeles,1,0.440,0.967,11,-5.830,0,0.2370,0.16300,...,0,0,0,0,0,0,0,0,0,0
4,Sleep Now In the Fire,The Battle Of Los Angeles,0,0.426,0.929,2,-6.729,1,0.0701,0.00162,...,0,0,0,0,0,0,0,0,0,0


In [73]:
df4.shape

(1126176, 36)

In [74]:
dropper = ['danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature']

In [75]:

df4.drop(columns = dropper, inplace = True)

In [76]:
df4.shape

(1126176, 26)

In [ ]:
df5 = pd.concat([df4, genre_df, scaled_float_df], axis = 1)